In [ ]:
import csv
import ijson
from datetime import datetime
import numpy as np
import pandas as pd


## import data

In [ ]:
with open('data/part_v003_o001_r_00000.json', 'r') as file:
    pet_like = ijson.items(file, 'text', multiple_values=True)
    for value in pet_like:
        print(value)

In [25]:
from datetime import datetime
date_dt1 = datetime.strptime(date_dt, '%a %b %d %H:%M:%S +0000 %Y')
print(date_dt1)

2020-02-29 14:57:32


In [59]:
# extract 'dates'
start = time.time()
data = ijson.parse(open('../data/part_v003_o001_r_00000.json', 'r'), multiple_values=True)

l_date = [datetime.strptime(value, '%a %b %d %H:%M:%S +0000 %Y') for prefix, event, value in data if prefix == 'created_at']

print ("elapsed_time:{0}".format(time.time() - start) + "[sec]")
slack_notify()

elapsed_time:1077.0222840309143[sec]


In [36]:
# extract 'tweets'
start = time.time()
data = ijson.parse(open('../data/part_v003_o001_r_00000.json', 'r'), multiple_values=True)

l_text = [value for prefix, event, value in data if prefix == 'text']

print ("elapsed_time:{0}".format(time.time() - start) + "[sec]")


elapsed_time:1124.0678129196167[sec]


In [656]:
print(len(l_date) == len(l_text))

True


## test 

In [490]:
date, twt = l_date.copy(), l_text.copy()
# date, twt = l_date[:100], l_text[:100]

In [491]:
a = np.vstack([date, twt]).T

In [492]:
df = pd.DataFrame(a,columns=['date', 'tweet'])

df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['hour'] = df['date'].dt.hour
# RT_flag: True if tweet containt 'RT', otherwise False
df['RT_flag'] = df['tweet'].str.contains('RT')

# df

In [493]:
# divide df into one with and without RT
dfrt = df[df['RT_flag'] == True]
dfor = df[df['RT_flag'] == False]

In [270]:
# # info of dataframe
# print(dfor.info(), 'length', len(dfor))
# print('           ')
# print(dfrt.info(), 'length', len(dfrt))

In [271]:
# # save to csv
# dfrt.to_csv('../dfs/df_rt_100.csv', index=False)
# dfor.to_csv('../dfs/df_or_100.csv', index=False)
# dfrt.to_csv('../dfs/df_100.csv', index=False)

## Use dfor(DataFrame without RT) as a default 

In [494]:
# check dataframe's nan value
dfor.isnull().values.any()

False

## read wordslist csv 

In [495]:
words = pd.read_csv('../data/WordTimeSeries.csv', encoding='utf-8').rename(columns={'Unnamed: 0': 'types',
                                                                                    'file name': 'file_name', 
                                                                                    'orignal form': 'orignal_form',
                                                                                    'English translation': 'English_translation'})
words.columns

Index(['types', 'file_name', 'orignal_form', 'English_translation'], dtype='object')

In [496]:
print('words types:',set(words.types.tolist()))
filetypes = ['T', 'D', 'A', 'V', 'F', 'C']

words types: {nan, '#Fatigue', '#Depression', '#Vigor', '#Tension', '#Confusion', '#Anger'}


In [659]:
filenames = [filename for filename in words.file_name.tolist() if 'C' in filename]
filenames

['C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'C15',
 'C16',
 'C17',
 'C18',
 'C19',
 'C20',
 'C21',
 'C22',
 'C23',
 'C24',
 'C25',
 'C26',
 'C27',
 'C28',
 'C29',
 'C30',
 'C31',
 'C32',
 'C33',
 'C34',
 'C35']

## DataFrame of each filetypes

### 以下, Cについてやってみる

In [497]:
# Drop nan and False in order to use query 
booleanDictionary = {True: 'TRUE', False: 'FALSE'}
words = words.replace(booleanDictionary)
words = words.dropna(how='all')

df_c = words.query('file_name.str.contains("C")', engine='python')
df_c.head()

,types,file_name,orignal_form,English_translation
118,#Confusion,C1,自信がない,unconfident
119,NaN,C2,グダグダ,mess
120,NaN,C3,唖然,stunned
121,NaN,C4,まとまらない,unfocused
122,NaN,C5,混同,mix-up


## extract data which includee keyword

In [498]:
keywords_jp = df_c.orignal_form.tolist()
df_c_query = dfor[dfor['tweet'].str.contains('|'.join(keyword_jp))]
df_c_query.head()

,date,tweet,year,month,day,hour,RT_flag
1,2020-02-29 14:59:58,今夜は久しぶりに \nお月様を見れましたが、\n少し雲がかかって\nボンヤリ月でした\n☁🌛...,2020,2,29,14,False
3,2020-02-29 14:59:58,不二先輩の誕生日で言いたいことたくさんあるけど、上手くまとまらないね😭😭,2020,2,29,14,False
106,2020-02-29 14:59:49,10ヶ月の間ほんとにお世話になりました&lt;(_ _)&gt; ミャーさんやごごさんの...,2020,2,29,14,False
124,2020-02-29 14:59:49,@k7ya ２のオープニングバトルは、全滅しないと先に進まないのですよー。\n何かやりようが...,2020,2,29,14,False
133,2020-02-29 14:59:48,推しのソロを聞いた時に想った事をいつか言葉に出来たらって思うけど、語彙力無さすぎて全然まとま...,2020,2,29,14,False


In [518]:
'''
keywords_jp: keyword exist in tweet
filenames: columns correspended to keyword 
'''
start = time.time()

for col, key in zip(filenames, keywords_jp):
    l = []
    for row in df_c_query.itertuples():
        if key in row.tweet:
            l.append(1)
        else:
            l.append(0)
    df_c_query[col] = l
    
print ("elapsed_time:{0}".format(time.time() - start) + "[sec]")
# save to csv 
# df_c_query.to_csv('c.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


elapsed_time:75.72259473800659[sec]


In [539]:
df_c_query.head()

TypeError: cannot do slice indexing on <class 'pandas.core.indexes.range.RangeIndex'> with these indexers [2020-02-29] of <class 'str'>

In [528]:
df_c_query['date1'] = df_c_query.date.dt.date

In [561]:
df1 = df_c_query.groupby('date1').get_group('2020-02-29')

In [551]:
df1['C1'].sum()

2548

In [619]:
sum(df_c_query['C2'])

8929

In [580]:
uni_date = df_c_query['date1'].tolist()
uni_date = sorted(set(uni_date), key=uni_date.index)
print(uni_date)

[datetime.date(2020, 2, 29), datetime.date(2020, 2, 28), datetime.date(2020, 2, 27), datetime.date(2020, 2, 26)]


In [627]:
filenames = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11',
       'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20',
       'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29',
       'C30', 'C31', 'C32', 'C33', 'C34', 'C35']

In [628]:
d = {}
for date in uni_date:
    dinter = {}
    for col in filenames:
        df1 = df_c_query.groupby('date1').get_group(date)
        dinter[col] = df1[col].sum()
    d[str(date)] =dinter

In [660]:
l = []
for date in uni_date:
    d = {}
    d['date'] = str(date)
    for col in filenames:
        df1 = df_c_query.groupby('date1').get_group(date)
        d[col] = df1[col].sum()
    l.append(d)

In [631]:
l[0]

{'date': '2020-02-29',
 'C1': 1980,
 'C2': 2624,
 'C3': 1253,
 'C4': 522,
 'C5': 1118,
 'C6': 1090,
 'C7': 984,
 'C8': 446,
 'C9': 745,
 'C10': 580,
 'C11': 2314,
 'C12': 495,
 'C13': 2084,
 'C14': 164,
 'C15': 2360,
 'C16': 186,
 'C17': 158,
 'C18': 268,
 'C19': 424,
 'C20': 509,
 'C21': 996,
 'C22': 538,
 'C23': 1223,
 'C24': 1821,
 'C25': 449,
 'C26': 1266,
 'C27': 430,
 'C28': 729,
 'C29': 321,
 'C30': 151,
 'C31': 53,
 'C32': 105,
 'C33': 166,
 'C34': 235,
 'C35': 261}

In [ ]:
index = ['2020-02-29', '2020-02-28', '2020-02-27', '2020-02-26']
col = filenames


In [592]:
tst = np.array(d['2020-02-29']).T

In [594]:
col = tst[0]

array(['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11',
       'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20',
       'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29',
       'C30', 'C31', 'C32', 'C33', 'C34', 'C35'], dtype='<U4')

In [637]:
cols = filenames
cols.insert(0, 'date')

In [646]:
dft = pd.DataFrame(columns=cols)
for row in l:
    dft = dft.append(row, ignore_index=True)

dft

,date,C1,C2,C3,C4,C5,C6,C7,C8,C9,...,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35
0,2020-02-29,1980,2624,1253,522,1118,1090,984,446,745,...,1266,430,729,321,151,53,105,166,235,261
1,2020-02-28,2548,2958,1627,707,1281,1397,1306,541,1164,...,1889,825,1070,427,240,53,168,278,288,371
2,2020-02-27,2504,3018,1163,790,1326,1586,1234,592,1062,...,1701,1026,858,369,218,74,148,239,288,342
3,2020-02-26,314,329,100,107,191,203,131,72,94,...,70,89,102,30,30,10,26,21,37,55
